In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
import json

# Utilities and Functions

## BLEU Calculation

In [3]:
from sacrebleu import BLEU
from evaluate import load

In [4]:
import warnings
import logging
import os
from transformers import logging as transformers_logging

# Suppress ALL HuggingFace warnings
transformers_logging.set_verbosity_error()
warnings.filterwarnings("ignore")  # This suppresses all warnings

# Suppress NLTK messages - try both methods
logging.getLogger().setLevel(logging.ERROR)

In [5]:
def calculate_metrics_batch(references,hypotheses,verbose= False):
    # Load metrics
    bleu = load('bleu')
    meteor = load("meteor")
    rouge = load("rouge")
    bertscore = load("bertscore")
    
    # Format references for metrics that expect a single reference
    single_refs = [ref_list[0] for ref_list in references]
    
    # Calculate BLEU
    bleu_score = bleu.compute(predictions=hypotheses, references=single_refs)
    
    # Calculate METEOR
    meteor_score = meteor.compute(
        predictions=hypotheses,
        references=single_refs
    )
    
    # Calculate ROUGE
    rouge_scores = rouge.compute(
        predictions=hypotheses,
        references=single_refs,
        rouge_types=["rouge1", "rouge2", "rougeL"]
    )
    
    bert_scores = bertscore.compute(
        predictions=hypotheses,
        references=single_refs,
        lang="ar",  # Set language to Arabic
        # model_type="aubmindlab/bert-large-arabertv02"  # Using ArabertV2, a specialized Arabic BERT model
        # Alternative models:
        # - "bert-base-multilingual-cased"  # If you want a multilingual model
        # - "CAMeL-Lab/bert-base-arabic-camelbert-da"  # Another good Arabic BERT
    )
    
    # Average BERTScore across all sentences
    bert_f1 = sum(bert_scores['f1']) / len(bert_scores['f1'])
    
    if verbose:
        # Return all scores
        return {
            'bleu': bleu_score,
            'meteor': meteor_score['meteor'],
            'rouge': {
                'rouge1': {
                    'precision': rouge_scores['rouge1_precision'],
                    'recall': rouge_scores['rouge1_recall'],
                    'f1': rouge_scores['rouge1']
                },
                'rouge2': {
                    'precision': rouge_scores['rouge2_precision'],
                    'recall': rouge_scores['rouge2_recall'],
                    'f1': rouge_scores['rouge2']
                },
                'rougeL': {
                    'precision': rouge_scores['rougeL_precision'],
                    'recall': rouge_scores['rougeL_recall'],
                    'f1': rouge_scores['rougeL']
                }
            },
            'bert_score': {
                'precision': sum(bert_scores['precision']) / len(bert_scores['precision']),
                'recall': sum(bert_scores['recall']) / len(bert_scores['recall']),
                'f1': bert_f1
            }
        }
    else:
        # Return only primary metrics commonly reported in literature
        return {
            'bleu': bleu_score['bleu'],
            'meteor': meteor_score['meteor'],
            'rouge_l_f1': rouge_scores['rougeL'],  # Most commonly reported ROUGE metric
            'bert_score_f1': bert_f1  # Most commonly reported BERTScore metric
        }

In [6]:
def read_jsonl(filename):
    data = []
    with open(filename, 'r') as f:
        for line in f:
            data.append(json.loads(line.strip()))
    return data

In [7]:
def is_mostly_arabic(text, threshold=0.9):
    if not text:
        return False
    
    # Define Arabic Unicode ranges
    arabic_ranges = [
        (0x0600, 0x06FF),   # Arabic
        (0x0750, 0x077F),   # Arabic Supplement
        (0xFB50, 0xFDFF),   # Arabic Presentation Forms-A
        (0xFE70, 0xFEFF),   # Arabic Presentation Forms-B
        (0x0660, 0x0669),   # Arabic-Indic digits
    ]
    
    # Count total characters (excluding whitespace)
    total_chars = len([c for c in text if not c.isspace()])
    if total_chars == 0:
        return False
    
    # Count Arabic characters
    arabic_count = 0
    for char in text:
        if char.isspace():
            continue
        char_code = ord(char)
        is_arabic = any(start <= char_code <= end for start, end in arabic_ranges)
        if is_arabic:
            arabic_count += 1
    
    # Calculate percentage
    arabic_percentage = arabic_count / total_chars
    
    return arabic_percentage >= threshold

In [8]:
def is_valid_abstract(abstract_dict):
    generated_abstract = abstract_dict['generated_abstract']
    # filter by ability to generate
    first_ten_words = [word for word in generated_abstract.split() if word][:15]
    not_able_to_generate_keywords = ['للأسف','التحويل','تشويه']
    if set(first_ten_words).intersection(set(not_able_to_generate_keywords)):
        return False
    # filter by length, set minimum to 30 words
    if len([word for word in generated_abstract.split() if word]) < 30:
        return False
    # filter by arabic letters majority
    if not is_mostly_arabic(generated_abstract):
        return False
    return True

In [9]:
def remove_arabic_abstrcats_duplicates(abstracts):
    unique_abstracts = []
    for abstract in abstracts:
        if abstract['original_abstract'] in [unique_abstract['original_abstract'] for unique_abstract in unique_abstracts]:
            continue
        unique_abstracts.append(abstract)
    return unique_abstracts

In [10]:
def get_common_abstracts(abstracts):
    # Create a set of original_abstracts that exist in all models
    common_originals = {}
    for model_abstracts in abstracts.values():
        original_abstracts = {abstract['original_abstract'] for abstract in model_abstracts}
        if not common_originals:
            common_originals = set(original_abstracts)
        else:
            common_originals = set.intersection(common_originals, original_abstracts)
    
    # Find intersection of all sets to get common original abstracts
    # common_originals = set.intersection(*original_abstracts_sets)
    
    # Filter abstracts to only include those with common original abstracts
    filtered_abstracts = {}
    for model_name, model_abstracts in abstracts.items():
        filtered_abstracts[model_name] = [
            abstract for abstract in model_abstracts 
            if abstract['original_abstract'] in common_originals
        ]
    
    return filtered_abstracts


## Descriptive statistics calculation

In [11]:
import re
import pandas as pd
import numpy as np

def get_text_statistics(text):
    """Calculate word, sentence, and sentence length statistics for a given text."""
    # Clean text
    text = text.strip()
    
    # Split into sentences (handle multiple punctuation marks)
    sentences = [s.strip() for s in re.split(r'[.!?]+', text) if s.strip()]
    
    # Get word counts for each sentence
    sentence_word_counts = [len(s.split()) for s in sentences]
    
    stats = {
        'word_count': sum(sentence_word_counts),
        'sentence_count': len(sentences),
        'avg_sentence_length': np.mean(sentence_word_counts) if sentence_word_counts else 0,
        'min_sentence_length': min(sentence_word_counts) if sentence_word_counts else 0,
        'max_sentence_length': max(sentence_word_counts) if sentence_word_counts else 0
    }
    
    return stats

def calculate_descriptive_statistics(filtered_abstracts):
    """Analyze and compare original and generated abstracts across models."""
    results = {}
    
    # First, analyze original abstracts (should be same across models)
    original_stats = []
    first_model = next(iter(filtered_abstracts.values()))
    for item in first_model:
        original_stats.append(get_text_statistics(item['original_abstract']))
    
    # Calculate original abstract statistics
    orig_df = pd.DataFrame(original_stats)
    results['original'] = {
        'word_count': {
            'min': orig_df['word_count'].min(),
            'max': orig_df['word_count'].max(),
            'avg': orig_df['word_count'].mean()
        },
        'sentence_count': {
            'min': orig_df['sentence_count'].min(),
            'max': orig_df['sentence_count'].max(),
            'avg': orig_df['sentence_count'].mean()
        },
        'sentence_length': {
            'min': orig_df['min_sentence_length'].min(),
            'max': orig_df['max_sentence_length'].max(),
            'avg': orig_df['avg_sentence_length'].mean()
        }
    }
    
    # Analyze generated abstracts for each model
    results['generated'] = {}
    for model_name, abstracts in filtered_abstracts.items():
        generated_stats = []
        for item in abstracts:
            generated_stats.append(get_text_statistics(item['generated_abstract']))
        
        gen_df = pd.DataFrame(generated_stats)
        results['generated'][model_name] = {
            'word_count': {
                'min': gen_df['word_count'].min(),
                'max': gen_df['word_count'].max(),
                'avg': gen_df['word_count'].mean()
            },
            'sentence_count': {
                'min': gen_df['sentence_count'].min(),
                'max': gen_df['sentence_count'].max(),
                'avg': gen_df['sentence_count'].mean()
            },
            'sentence_length': {
                'min': gen_df['min_sentence_length'].min(),
                'max': gen_df['max_sentence_length'].max(),
                'avg': gen_df['avg_sentence_length'].mean()
            }
        }
    
    return results

def print_discriptive_statistics_comparision(results):
    """Print a formatted comparison of the analysis results."""
    # Print original abstract statistics
    print("Original Abstract Statistics:")
    print("=" * 60)
    orig_stats = results['original']
    
    for metric, values in orig_stats.items():
        print(f"\n{metric.replace('_', ' ').title()}:")
        print(f"  Min: {values['min']:.1f}")
        print(f"  Max: {values['max']:.1f}")
        print(f"  Avg: {values['avg']:.1f}")
    
    # Print generated abstract statistics for each model
    print("\nGenerated Abstract Statistics:")
    print("=" * 60)
    
    for model_name, stats in results['generated'].items():
        print(f"\n{model_name}:")
        print("-" * 40)
        
        for metric, values in stats.items():
            print(f"\n{metric.replace('_', ' ').title()}:")
            orig_values = orig_stats[metric]
            
            print(f"  Min: {values['min']:.1f} (Δ: {values['min'] - orig_values['min']:+.1f})")
            print(f"  Max: {values['max']:.1f} (Δ: {values['max'] - orig_values['max']:+.1f})")
            print(f"  Avg: {values['avg']:.1f} (Δ: {values['avg'] - orig_values['avg']:+.1f})")

In [12]:
# save abstracts list as jsonl file
def save_jsonl(abstracts, filename):
    try:
        with open(filename, 'w', encoding='utf-8') as file:
            for abstract in abstracts:
                file.write(json.dumps(abstract, ensure_ascii=False) + '\n')
    except IOError as e:
        print(f"Error writing file: {e}")
    except json.JSONEncodeError as e:
        print(f"Error encoding JSON: {e}")

## From title abstracts

In [13]:
abstracts = dict(
    allam = read_jsonl('generated_arabic_datasets/allam/arabic_abstracts_dataset/from_title_abstracts_generation.jsonl'),
    openai = read_jsonl('generated_arabic_datasets/openai/arabic_abstracts_dataset/from_title_abstracts_generation.jsonl'),
    # claude = read_jsonl('generated_arabic_datasets/claude/arabic_abstracts_dataset/from_title_abstracts_generation.jsonl'),
    jais = read_jsonl('generated_arabic_datasets/jais-batched/arabic_abstracts_dataset/from_title_abstracts_generation.jsonl'),
    llama = read_jsonl('generated_arabic_datasets/llama-batched/arabic_abstracts_dataset/from_title_abstracts_generation.jsonl')
)

In [14]:
len(abstracts['allam']), len(abstracts['jais']), len(abstracts['llama']), len(abstracts['openai'])

(3000, 3000, 3000, 3000)

In [15]:
filtered_abstracts = {}

In [16]:
for model_name, model_abstracts in abstracts.items():
    filtered_abstracts[model_name] = remove_arabic_abstrcats_duplicates(model_abstracts)
len(filtered_abstracts['allam']), len(filtered_abstracts['jais']), len(filtered_abstracts['llama']), len(filtered_abstracts['openai'])

(2995, 2995, 2995, 2995)

filter abstracts

In [17]:
for model_name, abstracts in filtered_abstracts.items():
    filtered_abstracts[model_name] = []
    for abstract in abstracts:
        if is_valid_abstract(abstract):
            filtered_abstracts[model_name].append(abstract)
len(filtered_abstracts['allam']), len(filtered_abstracts['jais']), len(filtered_abstracts['llama']), len(filtered_abstracts['openai'])

(2991, 2976, 2986, 2991)

In [18]:
filtered_abstracts = get_common_abstracts(filtered_abstracts)

In [19]:
len(filtered_abstracts['allam']), len(filtered_abstracts['jais']), len(filtered_abstracts['llama']), len(filtered_abstracts['openai'])

(2963, 2963, 2963, 2963)

In [20]:
for model_name, model_abstracts in filtered_abstracts.items():
    filtered_abstracts[model_name] = remove_arabic_abstrcats_duplicates(model_abstracts)
len(filtered_abstracts['allam']), len(filtered_abstracts['jais']), len(filtered_abstracts['llama']), len(filtered_abstracts['openai'])

(2963, 2963, 2963, 2963)

In [21]:
len(filtered_abstracts['allam']), len(filtered_abstracts['jais']), len(filtered_abstracts['llama']), len(filtered_abstracts['openai'])

(2963, 2963, 2963, 2963)

In [22]:
sorted([abstract['original_abstract'] for abstract in filtered_abstracts['allam']])[:5],sorted([abstract['original_abstract'] for abstract in filtered_abstracts['allam']])[-5:]

(['"غني عن القول أنّ القرآن من هذه النّاحية حجاج صرف، لا لكونه خطابا حواريًّا فحسب، وإنّما لأنّه يدعو بصريح اللّفظ إلى نبذ العنف في شأن الإيمان، وهو ما يقتضي أنّه حوار وحجاج" قدور عمران. البعد التّداولي في الخطاب القرآني إسرائيل"الحكمة الإلهيّة في اختيار المعجزة من جنس ما اشتهر بين القوم هي أنّ الإنسان إذا أوتي من قِبل ما يعتبره مفخرته ومجال إجادته واعتزازه تكون الحجّة عليه أقوى والمعجز أكثر فعلا وأثرا". مصطفى مسلم. مباحث في إعجاز القرآن.',
  '(علم النص) آخر ما توصلت إليه اللسانيات الغربية المعاصرة، به انتقل البحث اللساني من نحو الجملة إلى نحو النص، وأصبح التحليل النصي للخطاب قائما على نظام الوحدة الموضوعية أو البنية الكبرى، وبها ترتبط الوحدات أو البنى الصغرى في النص..وقد أقام (علم النص) جملة من المعايير يحكم بها على نصية أو نصانية النص، ويزعم أصحاب هذا الفن أن لهم السبق فيما توصلوا إليه. فإن كان زعمهم يصدق عندهم في الغرب، فهل سبقوا حقا العرب في ذلك. وللإجابة عن هذا السؤال تأتي هذه المقاربة لواحد من العلوم الكثيرة عند العرب والمسلمين، هو التفسير الموضوعي للسورة القرآنية...',
  '.الاهتم

In [23]:
sorted([abstract['original_abstract'] for abstract in filtered_abstracts['jais']])[:5],sorted([abstract['original_abstract'] for abstract in filtered_abstracts['jais']])[-5:]

(['"غني عن القول أنّ القرآن من هذه النّاحية حجاج صرف، لا لكونه خطابا حواريًّا فحسب، وإنّما لأنّه يدعو بصريح اللّفظ إلى نبذ العنف في شأن الإيمان، وهو ما يقتضي أنّه حوار وحجاج" قدور عمران. البعد التّداولي في الخطاب القرآني إسرائيل"الحكمة الإلهيّة في اختيار المعجزة من جنس ما اشتهر بين القوم هي أنّ الإنسان إذا أوتي من قِبل ما يعتبره مفخرته ومجال إجادته واعتزازه تكون الحجّة عليه أقوى والمعجز أكثر فعلا وأثرا". مصطفى مسلم. مباحث في إعجاز القرآن.',
  '(علم النص) آخر ما توصلت إليه اللسانيات الغربية المعاصرة، به انتقل البحث اللساني من نحو الجملة إلى نحو النص، وأصبح التحليل النصي للخطاب قائما على نظام الوحدة الموضوعية أو البنية الكبرى، وبها ترتبط الوحدات أو البنى الصغرى في النص..وقد أقام (علم النص) جملة من المعايير يحكم بها على نصية أو نصانية النص، ويزعم أصحاب هذا الفن أن لهم السبق فيما توصلوا إليه. فإن كان زعمهم يصدق عندهم في الغرب، فهل سبقوا حقا العرب في ذلك. وللإجابة عن هذا السؤال تأتي هذه المقاربة لواحد من العلوم الكثيرة عند العرب والمسلمين، هو التفسير الموضوعي للسورة القرآنية...',
  '.الاهتم

In [24]:
sorted([abstract['original_abstract'] for abstract in filtered_abstracts['llama']])[:5],sorted([abstract['original_abstract'] for abstract in filtered_abstracts['llama']])[-5:]

(['"غني عن القول أنّ القرآن من هذه النّاحية حجاج صرف، لا لكونه خطابا حواريًّا فحسب، وإنّما لأنّه يدعو بصريح اللّفظ إلى نبذ العنف في شأن الإيمان، وهو ما يقتضي أنّه حوار وحجاج" قدور عمران. البعد التّداولي في الخطاب القرآني إسرائيل"الحكمة الإلهيّة في اختيار المعجزة من جنس ما اشتهر بين القوم هي أنّ الإنسان إذا أوتي من قِبل ما يعتبره مفخرته ومجال إجادته واعتزازه تكون الحجّة عليه أقوى والمعجز أكثر فعلا وأثرا". مصطفى مسلم. مباحث في إعجاز القرآن.',
  '(علم النص) آخر ما توصلت إليه اللسانيات الغربية المعاصرة، به انتقل البحث اللساني من نحو الجملة إلى نحو النص، وأصبح التحليل النصي للخطاب قائما على نظام الوحدة الموضوعية أو البنية الكبرى، وبها ترتبط الوحدات أو البنى الصغرى في النص..وقد أقام (علم النص) جملة من المعايير يحكم بها على نصية أو نصانية النص، ويزعم أصحاب هذا الفن أن لهم السبق فيما توصلوا إليه. فإن كان زعمهم يصدق عندهم في الغرب، فهل سبقوا حقا العرب في ذلك. وللإجابة عن هذا السؤال تأتي هذه المقاربة لواحد من العلوم الكثيرة عند العرب والمسلمين، هو التفسير الموضوعي للسورة القرآنية...',
  '.الاهتم

save filtered abstracts

In [25]:
# save_jsonl(filtered_abstracts['allam'], 'generated_arabic_datasets/allam/arabic_abstracts_dataset/from_title_abstracts_generation_filtered.jsonl')
# save_jsonl(filtered_abstracts['jais'], 'generated_arabic_datasets/jais-batched/arabic_abstracts_dataset/from_title_abstracts_generation_filtered.jsonl')
# save_jsonl(filtered_abstracts['llama'], 'generated_arabic_datasets/llama-batched/arabic_abstracts_dataset/from_title_abstracts_generation_filtered.jsonl')
# save_jsonl(filtered_abstracts['openai'], 'generated_arabic_datasets/openai/arabic_abstracts_dataset/from_title_abstracts_generation_filtered.jsonl')

print descriptive statistics

In [26]:
print_discriptive_statistics_comparision(calculate_descriptive_statistics(filtered_abstracts))

Original Abstract Statistics:

Word Count:
  Min: 75.0
  Max: 296.0
  Avg: 119.6

Sentence Count:
  Min: 1.0
  Max: 16.0
  Avg: 2.7

Sentence Length:
  Min: 1.0
  Max: 273.0
  Avg: 59.7

Generated Abstract Statistics:

allam:
----------------------------------------

Word Count:
  Min: 40.0 (Δ: -35.0)
  Max: 196.0 (Δ: -100.0)
  Avg: 77.2 (Δ: -42.4)

Sentence Count:
  Min: 2.0 (Δ: +1.0)
  Max: 14.0 (Δ: -2.0)
  Avg: 4.2 (Δ: +1.5)

Sentence Length:
  Min: 1.0 (Δ: +0.0)
  Max: 47.0 (Δ: -226.0)
  Avg: 18.6 (Δ: -41.1)

openai:
----------------------------------------

Word Count:
  Min: 98.0 (Δ: +23.0)
  Max: 143.0 (Δ: -153.0)
  Avg: 123.3 (Δ: +3.6)

Sentence Count:
  Min: 3.0 (Δ: +2.0)
  Max: 8.0 (Δ: -8.0)
  Avg: 5.2 (Δ: +2.6)

Sentence Length:
  Min: 1.0 (Δ: +0.0)
  Max: 53.0 (Δ: -220.0)
  Avg: 23.9 (Δ: -35.9)

jais:
----------------------------------------

Word Count:
  Min: 30.0 (Δ: -45.0)
  Max: 126.0 (Δ: -170.0)
  Avg: 62.3 (Δ: -57.3)

Sentence Count:
  Min: 1.0 (Δ: +0.0)
  Max: 9.0 (

calclate similarity metrics

In [27]:
for model_name, abstracts in filtered_abstracts.items():
    print(
        model_name, ':',
        calculate_metrics_batch(
            references=[[abstract['original_abstract']] for abstract in abstracts[:10]],
            hypotheses=[abstract['generated_abstract'] for abstract in abstracts[:10]]
        )
    )

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/majed_alshaibani/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/majed_alshaibani/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/majed_alshaibani/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


allam : {'bleu': 0.01799339323594296, 'meteor': 0.09366941226016803, 'rouge_l_f1': 0.03333333333333333, 'bert_score_f1': 0.6789880752563476}


[nltk_data] Downloading package wordnet to
[nltk_data]     /home/majed_alshaibani/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/majed_alshaibani/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/majed_alshaibani/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


openai : {'bleu': 0.016060202202749823, 'meteor': 0.1009792461228431, 'rouge_l_f1': 0.030769230769230764, 'bert_score_f1': 0.6818038165569306}


[nltk_data] Downloading package wordnet to
[nltk_data]     /home/majed_alshaibani/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/majed_alshaibani/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/majed_alshaibani/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


jais : {'bleu': 0.00974021868799892, 'meteor': 0.08282733000883899, 'rouge_l_f1': 0.02857142857142857, 'bert_score_f1': 0.6932152330875396}


[nltk_data] Downloading package wordnet to
[nltk_data]     /home/majed_alshaibani/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/majed_alshaibani/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/majed_alshaibani/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


llama : {'bleu': 0.023096491589497867, 'meteor': 0.102049822493532, 'rouge_l_f1': 0.02857142857142857, 'bert_score_f1': 0.6859988808631897}


# From title and content abstracts

In [28]:
abstracts = dict(
    allam = read_jsonl('generated_arabic_datasets/allam/arabic_abstracts_dataset/from_title_and_content_abstracts_generation.jsonl'),
    openai= read_jsonl('generated_arabic_datasets/openai/arabic_abstracts_dataset/from_title_and_content_abstracts_generation.jsonl'),
    # claude= json.load(open('generated_arabic_datasets/claude/arabic_abstracts_dataset/from_title_and_content_abstracts_generation.jsonl', 'r')),
    jais = read_jsonl('generated_arabic_datasets/jais-batched/arabic_abstracts_dataset/from_title_and_content_abstracts_generation.jsonl'),
    llama= read_jsonl('generated_arabic_datasets/llama-batched/arabic_abstracts_dataset/from_title_and_content_abstracts_generation.jsonl'),
)

In [29]:
len(abstracts['allam']), len(abstracts['jais']), len(abstracts['llama']), len(abstracts['openai'])

(3000, 3000, 3000, 3000)

filter abstracts

In [30]:
filtered_abstracts = {}

In [31]:
for model_name, model_abstracts in abstracts.items():
    filtered_abstracts[model_name] = remove_arabic_abstrcats_duplicates(model_abstracts)
len(filtered_abstracts['allam']), len(filtered_abstracts['jais']), len(filtered_abstracts['llama']), len(filtered_abstracts['openai'])

(2995, 2995, 2995, 2995)

In [32]:
for model_name, abstracts in filtered_abstracts.items():
    filtered_abstracts[model_name] = []
    for abstract in abstracts:
        if is_valid_abstract(abstract):
            filtered_abstracts[model_name].append(abstract)
len(filtered_abstracts['allam']), len(filtered_abstracts['jais']), len(filtered_abstracts['llama']), len(filtered_abstracts['openai'])

(2862, 2700, 2948, 2991)

In [33]:
filtered_abstracts = get_common_abstracts(filtered_abstracts)

In [34]:
len(filtered_abstracts['allam']), len(filtered_abstracts['jais']), len(filtered_abstracts['llama'])

(2574, 2574, 2574)

In [35]:
sorted([abstract['original_abstract'] for abstract in filtered_abstracts['allam']])[:5],sorted([abstract['original_abstract'] for abstract in filtered_abstracts['allam']])[-5:]

(['"غني عن القول أنّ القرآن من هذه النّاحية حجاج صرف، لا لكونه خطابا حواريًّا فحسب، وإنّما لأنّه يدعو بصريح اللّفظ إلى نبذ العنف في شأن الإيمان، وهو ما يقتضي أنّه حوار وحجاج" قدور عمران. البعد التّداولي في الخطاب القرآني إسرائيل"الحكمة الإلهيّة في اختيار المعجزة من جنس ما اشتهر بين القوم هي أنّ الإنسان إذا أوتي من قِبل ما يعتبره مفخرته ومجال إجادته واعتزازه تكون الحجّة عليه أقوى والمعجز أكثر فعلا وأثرا". مصطفى مسلم. مباحث في إعجاز القرآن.',
  '(علم النص) آخر ما توصلت إليه اللسانيات الغربية المعاصرة، به انتقل البحث اللساني من نحو الجملة إلى نحو النص، وأصبح التحليل النصي للخطاب قائما على نظام الوحدة الموضوعية أو البنية الكبرى، وبها ترتبط الوحدات أو البنى الصغرى في النص..وقد أقام (علم النص) جملة من المعايير يحكم بها على نصية أو نصانية النص، ويزعم أصحاب هذا الفن أن لهم السبق فيما توصلوا إليه. فإن كان زعمهم يصدق عندهم في الغرب، فهل سبقوا حقا العرب في ذلك. وللإجابة عن هذا السؤال تأتي هذه المقاربة لواحد من العلوم الكثيرة عند العرب والمسلمين، هو التفسير الموضوعي للسورة القرآنية...',
  '.الاهتم

In [36]:
sorted([abstract['original_abstract'] for abstract in filtered_abstracts['jais']])[:5],sorted([abstract['original_abstract'] for abstract in filtered_abstracts['jais']])[-5:]

(['"غني عن القول أنّ القرآن من هذه النّاحية حجاج صرف، لا لكونه خطابا حواريًّا فحسب، وإنّما لأنّه يدعو بصريح اللّفظ إلى نبذ العنف في شأن الإيمان، وهو ما يقتضي أنّه حوار وحجاج" قدور عمران. البعد التّداولي في الخطاب القرآني إسرائيل"الحكمة الإلهيّة في اختيار المعجزة من جنس ما اشتهر بين القوم هي أنّ الإنسان إذا أوتي من قِبل ما يعتبره مفخرته ومجال إجادته واعتزازه تكون الحجّة عليه أقوى والمعجز أكثر فعلا وأثرا". مصطفى مسلم. مباحث في إعجاز القرآن.',
  '(علم النص) آخر ما توصلت إليه اللسانيات الغربية المعاصرة، به انتقل البحث اللساني من نحو الجملة إلى نحو النص، وأصبح التحليل النصي للخطاب قائما على نظام الوحدة الموضوعية أو البنية الكبرى، وبها ترتبط الوحدات أو البنى الصغرى في النص..وقد أقام (علم النص) جملة من المعايير يحكم بها على نصية أو نصانية النص، ويزعم أصحاب هذا الفن أن لهم السبق فيما توصلوا إليه. فإن كان زعمهم يصدق عندهم في الغرب، فهل سبقوا حقا العرب في ذلك. وللإجابة عن هذا السؤال تأتي هذه المقاربة لواحد من العلوم الكثيرة عند العرب والمسلمين، هو التفسير الموضوعي للسورة القرآنية...',
  '.الاهتم

In [37]:
sorted([abstract['original_abstract'] for abstract in filtered_abstracts['llama']])[:5],sorted([abstract['original_abstract'] for abstract in filtered_abstracts['llama']])[-5:]

(['"غني عن القول أنّ القرآن من هذه النّاحية حجاج صرف، لا لكونه خطابا حواريًّا فحسب، وإنّما لأنّه يدعو بصريح اللّفظ إلى نبذ العنف في شأن الإيمان، وهو ما يقتضي أنّه حوار وحجاج" قدور عمران. البعد التّداولي في الخطاب القرآني إسرائيل"الحكمة الإلهيّة في اختيار المعجزة من جنس ما اشتهر بين القوم هي أنّ الإنسان إذا أوتي من قِبل ما يعتبره مفخرته ومجال إجادته واعتزازه تكون الحجّة عليه أقوى والمعجز أكثر فعلا وأثرا". مصطفى مسلم. مباحث في إعجاز القرآن.',
  '(علم النص) آخر ما توصلت إليه اللسانيات الغربية المعاصرة، به انتقل البحث اللساني من نحو الجملة إلى نحو النص، وأصبح التحليل النصي للخطاب قائما على نظام الوحدة الموضوعية أو البنية الكبرى، وبها ترتبط الوحدات أو البنى الصغرى في النص..وقد أقام (علم النص) جملة من المعايير يحكم بها على نصية أو نصانية النص، ويزعم أصحاب هذا الفن أن لهم السبق فيما توصلوا إليه. فإن كان زعمهم يصدق عندهم في الغرب، فهل سبقوا حقا العرب في ذلك. وللإجابة عن هذا السؤال تأتي هذه المقاربة لواحد من العلوم الكثيرة عند العرب والمسلمين، هو التفسير الموضوعي للسورة القرآنية...',
  '.الاهتم

save filtered abstracts

In [38]:
# save_jsonl(filtered_abstracts['allam'], 'generated_arabic_datasets/allam/arabic_abstracts_dataset/from_title_and_content_abstracts_generation_filtered.jsonl')
# save_jsonl(filtered_abstracts['jais'], 'generated_arabic_datasets/jais-batched/arabic_abstracts_dataset/from_title_and_content_abstracts_generation_filtered.jsonl')
# save_jsonl(filtered_abstracts['llama'], 'generated_arabic_datasets/llama-batched/arabic_abstracts_dataset/from_title_and_content_abstracts_generation_filtered.jsonl')
# save_jsonl(filtered_abstracts['openai'], 'generated_arabic_datasets/openai/arabic_abstracts_dataset/from_title_and_content_abstracts_generation_filtered.jsonl')

print descriptive statistics

In [39]:
print_discriptive_statistics_comparision(calculate_descriptive_statistics(filtered_abstracts))

Original Abstract Statistics:

Word Count:
  Min: 75.0
  Max: 296.0
  Avg: 119.7

Sentence Count:
  Min: 1.0
  Max: 16.0
  Avg: 2.7

Sentence Length:
  Min: 1.0
  Max: 273.0
  Avg: 59.4

Generated Abstract Statistics:

allam:
----------------------------------------

Word Count:
  Min: 30.0 (Δ: -45.0)
  Max: 2022.0 (Δ: +1726.0)
  Avg: 95.3 (Δ: -24.3)

Sentence Count:
  Min: 2.0 (Δ: +1.0)
  Max: 64.0 (Δ: +48.0)
  Avg: 5.5 (Δ: +2.9)

Sentence Length:
  Min: 1.0 (Δ: +0.0)
  Max: 133.0 (Δ: -140.0)
  Avg: 17.5 (Δ: -42.0)

openai:
----------------------------------------

Word Count:
  Min: 85.0 (Δ: +10.0)
  Max: 143.0 (Δ: -153.0)
  Avg: 113.9 (Δ: -5.7)

Sentence Count:
  Min: 3.0 (Δ: +2.0)
  Max: 9.0 (Δ: -7.0)
  Avg: 5.3 (Δ: +2.7)

Sentence Length:
  Min: 1.0 (Δ: +0.0)
  Max: 65.0 (Δ: -208.0)
  Avg: 21.7 (Δ: -37.7)

jais:
----------------------------------------

Word Count:
  Min: 30.0 (Δ: -45.0)
  Max: 435.0 (Δ: +139.0)
  Avg: 105.7 (Δ: -13.9)

Sentence Count:
  Min: 1.0 (Δ: +0.0)
  Max: 

calclate similarity metrics

In [40]:
for model_name, abstracts in filtered_abstracts.items():
    print(
        model_name, ':',
        calculate_metrics_batch(
            references=[[abstract['original_abstract']] for abstract in abstracts],
            hypotheses=[abstract['generated_abstract'] for abstract in abstracts]
        )
    )

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/majed_alshaibani/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/majed_alshaibani/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/majed_alshaibani/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


allam : {'bleu': 0.11746364398452952, 'meteor': 0.2531873825557477, 'rouge_l_f1': 0.14014168034870622, 'bert_score_f1': 0.7552560666270652}


[nltk_data] Downloading package wordnet to
[nltk_data]     /home/majed_alshaibani/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/majed_alshaibani/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/majed_alshaibani/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


openai : {'bleu': 0.06058084721504066, 'meteor': 0.1943649888565276, 'rouge_l_f1': 0.15752823674717972, 'bert_score_f1': 0.7399400753528459}


[nltk_data] Downloading package wordnet to
[nltk_data]     /home/majed_alshaibani/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/majed_alshaibani/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/majed_alshaibani/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


jais : {'bleu': 0.12207960279313508, 'meteor': 0.25997114162235313, 'rouge_l_f1': 0.14702779090178136, 'bert_score_f1': 0.7573456475576321}


[nltk_data] Downloading package wordnet to
[nltk_data]     /home/majed_alshaibani/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/majed_alshaibani/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/majed_alshaibani/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


llama : {'bleu': 0.14018576129998128, 'meteor': 0.2718974679277035, 'rouge_l_f1': 0.1214896184456037, 'bert_score_f1': 0.7570831812484003}


# Polished abstracts

In [41]:
abstracts = dict(
    allam = read_jsonl('generated_arabic_datasets/allam/arabic_abstracts_dataset/by_polishing_abstracts_abstracts_generation.jsonl'),
    openai= read_jsonl('generated_arabic_datasets/openai/arabic_abstracts_dataset/by_polishing_abstracts_abstracts_generation.jsonl'),
    # claude= json.load(open('generated_arabic_datasets/claude/arabic_abstracts_dataset/by_polishing_abstracts_abstracts_generation.jsonl', 'r')),
    jais = read_jsonl('generated_arabic_datasets/jais-batched/arabic_abstracts_dataset/by_polishing_abstracts_abstracts_generation.jsonl'),
    llama= read_jsonl('generated_arabic_datasets/llama-batched/arabic_abstracts_dataset/by_polishing_abstracts_abstracts_generation.jsonl'),
)

In [42]:
len(abstracts['allam']), len(abstracts['jais']), len(abstracts['llama']), len(abstracts['openai'])

(3000, 3000, 3000, 3000)

filter abstracts

In [43]:
filtered_abstracts = {}

In [44]:
for model_name, model_abstracts in abstracts.items():
    filtered_abstracts[model_name] = remove_arabic_abstrcats_duplicates(model_abstracts)
len(filtered_abstracts['allam']), len(filtered_abstracts['jais']), len(filtered_abstracts['llama']), len(filtered_abstracts['openai'])

(2995, 2995, 2995, 2995)

In [45]:
for model_name, abstracts in filtered_abstracts.items():
    filtered_abstracts[model_name] = []
    for abstract in abstracts:
        if is_valid_abstract(abstract):
            filtered_abstracts[model_name].append(abstract)
len(filtered_abstracts['allam']), len(filtered_abstracts['jais']), len(filtered_abstracts['llama'])

(2989, 2899, 2980)

In [46]:
filtered_abstracts = get_common_abstracts(filtered_abstracts)

In [47]:
len(filtered_abstracts['allam']), len(filtered_abstracts['jais']), len(filtered_abstracts['llama'])

(2851, 2851, 2851)

In [48]:
sorted([abstract['original_abstract'] for abstract in filtered_abstracts['allam']])[:5],sorted([abstract['original_abstract'] for abstract in filtered_abstracts['allam']])[-5:]

(['"غني عن القول أنّ القرآن من هذه النّاحية حجاج صرف، لا لكونه خطابا حواريًّا فحسب، وإنّما لأنّه يدعو بصريح اللّفظ إلى نبذ العنف في شأن الإيمان، وهو ما يقتضي أنّه حوار وحجاج" قدور عمران. البعد التّداولي في الخطاب القرآني إسرائيل"الحكمة الإلهيّة في اختيار المعجزة من جنس ما اشتهر بين القوم هي أنّ الإنسان إذا أوتي من قِبل ما يعتبره مفخرته ومجال إجادته واعتزازه تكون الحجّة عليه أقوى والمعجز أكثر فعلا وأثرا". مصطفى مسلم. مباحث في إعجاز القرآن.',
  '(علم النص) آخر ما توصلت إليه اللسانيات الغربية المعاصرة، به انتقل البحث اللساني من نحو الجملة إلى نحو النص، وأصبح التحليل النصي للخطاب قائما على نظام الوحدة الموضوعية أو البنية الكبرى، وبها ترتبط الوحدات أو البنى الصغرى في النص..وقد أقام (علم النص) جملة من المعايير يحكم بها على نصية أو نصانية النص، ويزعم أصحاب هذا الفن أن لهم السبق فيما توصلوا إليه. فإن كان زعمهم يصدق عندهم في الغرب، فهل سبقوا حقا العرب في ذلك. وللإجابة عن هذا السؤال تأتي هذه المقاربة لواحد من العلوم الكثيرة عند العرب والمسلمين، هو التفسير الموضوعي للسورة القرآنية...',
  '.الاهتم

In [49]:
sorted([abstract['original_abstract'] for abstract in filtered_abstracts['jais']])[:5],sorted([abstract['original_abstract'] for abstract in filtered_abstracts['jais']])[-5:]

(['"غني عن القول أنّ القرآن من هذه النّاحية حجاج صرف، لا لكونه خطابا حواريًّا فحسب، وإنّما لأنّه يدعو بصريح اللّفظ إلى نبذ العنف في شأن الإيمان، وهو ما يقتضي أنّه حوار وحجاج" قدور عمران. البعد التّداولي في الخطاب القرآني إسرائيل"الحكمة الإلهيّة في اختيار المعجزة من جنس ما اشتهر بين القوم هي أنّ الإنسان إذا أوتي من قِبل ما يعتبره مفخرته ومجال إجادته واعتزازه تكون الحجّة عليه أقوى والمعجز أكثر فعلا وأثرا". مصطفى مسلم. مباحث في إعجاز القرآن.',
  '(علم النص) آخر ما توصلت إليه اللسانيات الغربية المعاصرة، به انتقل البحث اللساني من نحو الجملة إلى نحو النص، وأصبح التحليل النصي للخطاب قائما على نظام الوحدة الموضوعية أو البنية الكبرى، وبها ترتبط الوحدات أو البنى الصغرى في النص..وقد أقام (علم النص) جملة من المعايير يحكم بها على نصية أو نصانية النص، ويزعم أصحاب هذا الفن أن لهم السبق فيما توصلوا إليه. فإن كان زعمهم يصدق عندهم في الغرب، فهل سبقوا حقا العرب في ذلك. وللإجابة عن هذا السؤال تأتي هذه المقاربة لواحد من العلوم الكثيرة عند العرب والمسلمين، هو التفسير الموضوعي للسورة القرآنية...',
  '.الاهتم

In [50]:
sorted([abstract['original_abstract'] for abstract in filtered_abstracts['llama']])[:5],sorted([abstract['original_abstract'] for abstract in filtered_abstracts['llama']])[-5:]

(['"غني عن القول أنّ القرآن من هذه النّاحية حجاج صرف، لا لكونه خطابا حواريًّا فحسب، وإنّما لأنّه يدعو بصريح اللّفظ إلى نبذ العنف في شأن الإيمان، وهو ما يقتضي أنّه حوار وحجاج" قدور عمران. البعد التّداولي في الخطاب القرآني إسرائيل"الحكمة الإلهيّة في اختيار المعجزة من جنس ما اشتهر بين القوم هي أنّ الإنسان إذا أوتي من قِبل ما يعتبره مفخرته ومجال إجادته واعتزازه تكون الحجّة عليه أقوى والمعجز أكثر فعلا وأثرا". مصطفى مسلم. مباحث في إعجاز القرآن.',
  '(علم النص) آخر ما توصلت إليه اللسانيات الغربية المعاصرة، به انتقل البحث اللساني من نحو الجملة إلى نحو النص، وأصبح التحليل النصي للخطاب قائما على نظام الوحدة الموضوعية أو البنية الكبرى، وبها ترتبط الوحدات أو البنى الصغرى في النص..وقد أقام (علم النص) جملة من المعايير يحكم بها على نصية أو نصانية النص، ويزعم أصحاب هذا الفن أن لهم السبق فيما توصلوا إليه. فإن كان زعمهم يصدق عندهم في الغرب، فهل سبقوا حقا العرب في ذلك. وللإجابة عن هذا السؤال تأتي هذه المقاربة لواحد من العلوم الكثيرة عند العرب والمسلمين، هو التفسير الموضوعي للسورة القرآنية...',
  '.الاهتم

save filetered abstracts

In [51]:
# save_jsonl(filtered_abstracts['allam'], 'generated_arabic_datasets/allam/arabic_abstracts_dataset/by_polishing_abstracts_abstracts_generation_filtered.jsonl')
# save_jsonl(filtered_abstracts['jais'], 'generated_arabic_datasets/jais-batched/arabic_abstracts_dataset/by_polishing_abstracts_abstracts_generation_filtered.jsonl')
# save_jsonl(filtered_abstracts['llama'], 'generated_arabic_datasets/llama-batched/arabic_abstracts_dataset/by_polishing_abstracts_abstracts_generation_filtered.jsonl')
# save_jsonl(filtered_abstracts['openai'], 'generated_arabic_datasets/openai/arabic_abstracts_dataset/by_polishing_abstracts_abstracts_generation_filtered.jsonl')

print descriptive statistics

In [52]:
print_discriptive_statistics_comparision(calculate_descriptive_statistics(filtered_abstracts))

Original Abstract Statistics:

Word Count:
  Min: 75.0
  Max: 292.0
  Avg: 119.7

Sentence Count:
  Min: 1.0
  Max: 16.0
  Avg: 2.6

Sentence Length:
  Min: 1.0
  Max: 273.0
  Avg: 60.1

Generated Abstract Statistics:

allam:
----------------------------------------

Word Count:
  Min: 41.0 (Δ: -34.0)
  Max: 2796.0 (Δ: +2504.0)
  Avg: 104.2 (Δ: -15.5)

Sentence Count:
  Min: 2.0 (Δ: +1.0)
  Max: 160.0 (Δ: +144.0)
  Avg: 5.3 (Δ: +2.6)

Sentence Length:
  Min: 1.0 (Δ: +0.0)
  Max: 83.0 (Δ: -190.0)
  Avg: 20.4 (Δ: -39.7)

openai:
----------------------------------------

Word Count:
  Min: 86.0 (Δ: +11.0)
  Max: 224.0 (Δ: -68.0)
  Avg: 165.1 (Δ: +45.3)

Sentence Count:
  Min: 4.0 (Δ: +3.0)
  Max: 11.0 (Δ: -5.0)
  Avg: 6.4 (Δ: +3.7)

Sentence Length:
  Min: 1.0 (Δ: +0.0)
  Max: 88.0 (Δ: -185.0)
  Avg: 26.4 (Δ: -33.7)

jais:
----------------------------------------

Word Count:
  Min: 30.0 (Δ: -45.0)
  Max: 197.0 (Δ: -95.0)
  Avg: 68.5 (Δ: -51.2)

Sentence Count:
  Min: 1.0 (Δ: +0.0)
  Max:

calclate similarity metrics

In [53]:
for model_name, abstracts in filtered_abstracts.items():
    print(
        model_name, ':',
        calculate_metrics_batch(
            references=[[abstract['original_abstract']] for abstract in abstracts],
            hypotheses=[abstract['generated_abstract'] for abstract in abstracts]
        )
    )

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/majed_alshaibani/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/majed_alshaibani/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/majed_alshaibani/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


allam : {'bleu': 0.15636921757110714, 'meteor': 0.32941947819011563, 'rouge_l_f1': 0.23642266869812717, 'bert_score_f1': 0.7879079615848184}


[nltk_data] Downloading package wordnet to
[nltk_data]     /home/majed_alshaibani/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/majed_alshaibani/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/majed_alshaibani/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


openai : {'bleu': 0.063989422395284, 'meteor': 0.24653341877483895, 'rouge_l_f1': 0.24189900797616912, 'bert_score_f1': 0.755879987649272}


[nltk_data] Downloading package wordnet to
[nltk_data]     /home/majed_alshaibani/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/majed_alshaibani/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/majed_alshaibani/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


jais : {'bleu': 0.08806314413537457, 'meteor': 0.244176863302324, 'rouge_l_f1': 0.21660188222032023, 'bert_score_f1': 0.7765476241608495}


[nltk_data] Downloading package wordnet to
[nltk_data]     /home/majed_alshaibani/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/majed_alshaibani/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/majed_alshaibani/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


llama : {'bleu': 0.20419142720821493, 'meteor': 0.3758039050592304, 'rouge_l_f1': 0.22407347944888933, 'bert_score_f1': 0.8063716400468278}


In [ ]:
exit()

: 